In [81]:
import os
import sys

import pandas as pd
import geopandas as gpd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
from geopy import geocoders
import time
import numpy as np

In [188]:

def get_location_by_address(address):
    """This function returns a location as raw from an address
    will repeat until success"""
    geolocator = geocoders.GoogleV3(api_key="") #INSERT API KEY HERE
    location = geolocator.geocode(address, timeout = 10)
    if location is not None:
        return location
    else:
        test = {'latitude': np.nan, 'longitude': np.nan}
        ser = pd.Series(test)
        return ser

In [82]:
df = pd.read_csv(f'/home/jovyan/work/data/external/sigade/sigade_full.csv')
# Downloads municipality polygon according to code
#query = f"SELECT * FROM marco.mpos_2020 WHERE \"CVE_ENT\" LIKE \'{edo}\'"
#mun_gdf = mun_gdf.append(aup.gdf_from_query(query, geometry_col='geometry'))
#poly_wkt = mun_gdf.dissolve().geometry.to_wkt()[0]
### Read schools
df.columns= df.columns.str.lower()
df.columns = df.columns.str.replace(" ", "_")
df['clave_de_la_entidad_federativa'] = df['clave_de_la_entidad_federativa'].fillna(0)
df['clave_del_municipio_o_delegación'] = df['clave_del_municipio_o_delegación'].fillna(0)
df = df.astype({"clave_de_la_entidad_federativa": int}, errors='raise') 
df = df.astype({"clave_del_municipio_o_delegación": int}, errors='raise') 
for column in df.columns[df.dtypes=='object']:
    df[column] = df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df['x'] = df['ubicación_de_la_escuela-localidad_al_oeste_del_meridiano_de_greenwich,_expresada_en_grados']
df['y'] = df['ubicación_de_la_escuela-localidad_al_norte_del_ecuador,_expresada_en_grados']
for idx,row in df.iterrows():
    if df.loc[idx, 'x']>=0:
        df.loc[idx, 'x'] = df.loc[idx, 'x']*-1
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y), crs="EPSG:4326" )
gdf["CVEGEOMUN"] = gdf.apply(
    lambda row: "{:02d}".format((row.loc["clave_de_la_entidad_federativa"]))
    + "{:03d}".format((row.loc["clave_del_municipio_o_delegación"])),
    axis=1,
    )
print('code done')
gdf = gdf.drop_duplicates(subset=['clave_del_centro_de_trabajo'], keep = 'first')
gdf = gdf.set_index('clave_del_centro_de_trabajo')
gdf['número_exterior'] = gdf['número_exterior'].astype(str)
gdf['código_postal'] = gdf['código_postal'].astype(str)
gdf['address'] = (gdf['domicilio'] + ' ' + gdf['número_exterior'] +  ', ' + 
    gdf['nombre_del_municipio_o_delegación'] + ', ' + gdf['nombre_de_la_entidad']
     + ', ' + gdf['código_postal'] + ', MEXICO')


code done


In [96]:
len(gdf)

274320

In [ ]:
### Saves raw geodataframe to file
#gdf.to_file('sigade_all_raw.geojson')

In [134]:
cities = pd.read_json('/home/jovyan/work/scripts/Metropolis_CVE.json')
wrong_sigade = gpd.GeoDataFrame()
good_sigade = gpd.GeoDataFrame()
all_sigade_city = gpd.GeoDataFrame()
for c in cities.columns.unique():
    aup.log(f"{c} starting")
    query = f"SELECT * FROM metropolis.metro_list WHERE \"city\" LIKE \'{c}\'"
    mun_gdf = aup.gdf_from_query(query, geometry_col='geometry')
    # Iterates over municipality codes for each metropolitan area or capital
    sigade_city = gdf[gdf['CVEGEOMUN'].isin(mun_gdf.CVEGEO)]
    all_sigade_city = all_sigade_city.append(sigade_city)
    ##CHECK IF POINTS ARE WITHIN MUNICIPALITIES
    mun_poly = mun_gdf.unary_union
    within_mun = sigade_city[sigade_city.geometry.within(mun_poly)]
    wrong_sigade =wrong_sigade.append(sigade_city[~sigade_city.geometry.within(mun_poly)])
    ##CHECK FOR DUPLICATE COORDINATES WHICH MAY INDICATE WRONG LOCATION
    good_sigade = good_sigade.append(within_mun.drop_duplicates(subset=['geometry'], keep = False))
    wrong_sigade = wrong_sigade.append(within_mun[within_mun.duplicated(subset=['geometry'], keep = False)])


In [135]:
len(all_sigade_city)

110321

In [137]:
len(wrong_sigade)

67702

In [138]:
len(good_sigade)

42619

In [139]:
good_sigade.to_file('good_sigade_all.geojson')
wrong_sigade.to_file('wrong_sigade_all.geojson')

In [ ]:
wrong = gpd.read_file('/home/jovyan/work/notebooks/wrong_sigade_all.geojson')

In [ ]:
good = gpd.read_file('/home/jovyan/work/notebooks/good_sigade_all.geojson')

In [146]:
wrong = wrong_sigade.copy()

In [150]:
wrong['domicilio'] = wrong['domicilio'].astype(str)
missingvalid = wrong[~wrong['domicilio'].str.contains('NINGUNO')]
missingvalid = missingvalid[~missingvalid['domicilio'].str.contains('None')]

In [151]:
print(str(len(missingvalid)) + ' out of a total: ' + str(len(wrong)))

62766 out of a total: 67702


In [156]:
missingvalid.drop(['geometry'], axis=1, inplace=True)
wrong_part1 = missingvalid.iloc[:25000]
wrong_part2 = missingvalid.iloc[25000:]

In [ ]:
missing_part1 = wrong_part1[wrong_part1['latitud'].isna()]

In [189]:
count = 0
good = 0
for idx,row in wrong_part2.iterrows():
    address = wrong_part2.loc[idx, 'address']
    location = get_location_by_address(address)
    latitude = location.latitude
    longitude = location.longitude
    if pd.notna(latitude):
        good = good + 1
        missingvalid.loc[idx, 'latitud'] = latitude
        missingvalid.loc[idx, 'longitud'] = longitude
        wrong_part2.loc[idx, 'latitud'] = latitude
        wrong_part2.loc[idx, 'longitud'] = longitude
    count = count +1
    print(str(count) + '/' + str(len(wrong_part2)))

1/37766
2/37766
3/37766
4/37766
5/37766
6/37766
7/37766
8/37766
9/37766
10/37766
11/37766
12/37766
13/37766
14/37766
15/37766
16/37766
17/37766
18/37766
19/37766
20/37766
21/37766
22/37766
23/37766
24/37766
25/37766
26/37766
27/37766
28/37766
29/37766
30/37766
31/37766
32/37766
33/37766
34/37766
35/37766
36/37766
37/37766
38/37766
39/37766
40/37766
41/37766
42/37766
43/37766
44/37766
45/37766
46/37766
47/37766
48/37766
49/37766
50/37766
51/37766
52/37766
53/37766
54/37766
55/37766
56/37766
57/37766
58/37766
59/37766
60/37766
61/37766
62/37766
63/37766
64/37766
65/37766
66/37766
67/37766
68/37766
69/37766
70/37766
71/37766
72/37766
73/37766
74/37766
75/37766
76/37766
77/37766
78/37766
79/37766
80/37766
81/37766
82/37766
83/37766
84/37766
85/37766
86/37766
87/37766
88/37766
89/37766
90/37766
91/37766
92/37766
93/37766
94/37766
95/37766
96/37766
97/37766
98/37766
99/37766
100/37766
101/37766
102/37766
103/37766
104/37766
105/37766
106/37766
107/37766
108/37766
109/37766
110/37766
111/3776

In [190]:
len(missingvalid[missingvalid['latitud'].isna()])

8

In [192]:
missing_extra = missingvalid[missingvalid['latitud'].isna()]

In [194]:
corrected_all = missingvalid[missingvalid['latitud'].notna()]

In [196]:
corrected_all.to_csv('sigade_corrected_all.csv')

In [198]:
corrected = gpd.GeoDataFrame(corrected_all,geometry=gpd.points_from_xy(corrected_all.longitud, corrected_all.latitud))
corrected = corrected.set_crs("EPSG:4326")

In [199]:
corrected

,unnamed:_0,clave_del_turno,nombre_del_turno,nombre_del_centro_de_trabajo,tipo_educativo,nivel_educativo,servicio_educativo,nombre_del_control_(público_o_privado),tipo_de_sostenimiento,clave_de_la_entidad_federativa,...,"ubicación_de_la_escuela-localidad_al_norte_del_ecuador,_expresada_en_grados,_minutos_y_segundos","ubicación_de_la_escuela-localidad_al_oeste_del_meridiano_de_greenwich,_expresada_en_grados","ubicación_de_la_escuela-localidad_al_norte_del_ecuador,_expresada_en_grados",x,y,CVEGEOMUN,address,latitud,longitud,geometry
clave_del_centro_de_trabajo,,,,,,,,,,,,,,,,,,,,,
01PDI0066X,282,1,MATUTINO,CENTRO DE ATENCION INFANTIL BEBE FELIZ SC,INICIAL,INICIAL,LACTANTE Y MATERNAL,PRIVADO,PRIVADO,1,...,0:54:53.22 N,102.341944,0.914782,-102.341944,0.914782,01005,"AVENIDA GUADALUPE 614, JESUS MARIA, AGUASCALIE...",21.952413,-102.334240,POINT (-102.33424 21.95241)
01KJN0214Q,551,1,MATUTINO,PREESCOLAR COMUNITARIO,BASICA,INICIAL GENERAL,INICIAL INDIGENA,PUBLICO,FEDERAL,1,...,0:55:1.17 N,102.198056,0.916991,-102.198056,0.916991,01001,"CALLE SAN DAVID GALVAN 0, AGUASCALIENTES, AGUA...",21.844640,-102.270188,POINT (-102.27019 21.84464)
01ODI0001O,639,1,MATUTINO,ESTANCIA DE BIENESTAR INFANTIL DE ISSSSPEA,INICIAL,INICIAL,LACTANTE Y MATERNAL,PUBLICO,ESTATAL,1,...,1:52:6.91 N,-102.272700,1.868585,-102.272700,1.868585,01001,"CALLE UNO 101, AGUASCALIENTES, AGUASCALIENTES,...",21.873790,-102.260283,POINT (-102.26028 21.87379)
01PBH3818C,2013,2,VESPERTINO,BACHILLERATO UNEA CAMPUS AGUASCALIENTES,MEDIA SUPERIOR,BACHILLERATO,BACHILLERATO GENERAL,PRIVADO,PRIVADO,1,...,21:31:27.84 N,-102.180000,21.524400,-102.180000,21.524400,01001,"AVENIDA HEROE DE NACOZARI SUR 402, AGUASCALIEN...",21.877946,-102.281301,POINT (-102.28130 21.87795)
01ETC0015H,2303,2,VESPERTINO,COLEGIO DE ESTUDIOS CIENTIFICOS Y TECNOLOGICOS...,MEDIA SUPERIOR,BACHILLERATO,TECNOLOGICO,PUBLICO,ESTATAL,1,...,21:29:35.16 N,-102.111600,21.493100,-102.111600,21.493100,01001,"AVENIDA REVOLUCION 0, AGUASCALIENTES, AGUASCAL...",21.847463,-102.314782,POINT (-102.31478 21.84746)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32PSU0022Q,5567,4,DISCONTINUO,UNIVERSIDAD INTERAMERICANA PARA EL DESARROLLO ...,SUPERIOR,LICENCIATURA,LICENCIATURA UNIVERSITARIA Y TECNOLOGICA,PRIVADO,PRIVADO,32,...,22:44:48.48 N,-102.519000,22.746800,-102.519000,22.746800,32017,"CALLE VIALIDAD SAN SIMON 11, GUADALUPE, ZACATE...",22.767834,-102.515261,POINT (-102.51526 22.76783)
32PBH0003H,5570,1,MATUTINO,JUANA DE ARCO,MEDIA SUPERIOR,BACHILLERATO,BACHILLERATO GENERAL,PRIVADO,PRIVADO,32,...,22:46:12.19 N,-102.566466,22.770054,-102.566466,22.770054,32056,"CALLE CARRETERA A LA BUFA 0, ZACATECAS, ZACATE...",22.772187,-102.559574,POINT (-102.55957 22.77219)
32DPN0001S,5571,4,DISCONTINUO,UNIDAD PROFESIONAL INTERDISCIPLINARIA DE INGEN...,SUPERIOR,LICENCIATURA,LICENCIATURA UNIVERSITARIA Y TECNOLOGICA,PUBLICO,FEDERAL,32,...,22:46:33.96 N,-102.572000,22.776100,-102.572000,22.776100,32056,"BOULEVARD EL BOTE 0, ZACATECAS, ZACATECAS, 981...",22.776989,-102.603417,POINT (-102.60342 22.77699)


In [211]:
good_sigade['latitud'] = good_sigade['y'].copy()
good_sigade['longitud'] = good_sigade['x'].copy()
good_sigade.drop(['geometry'], axis =1, inplace=True)
good_final = gpd.GeoDataFrame(good_sigade,geometry=gpd.points_from_xy(good_sigade.longitud, good_sigade.latitud))

,unnamed:_0,clave_del_turno,nombre_del_turno,nombre_del_centro_de_trabajo,tipo_educativo,nivel_educativo,servicio_educativo,nombre_del_control_(público_o_privado),tipo_de_sostenimiento,clave_de_la_entidad_federativa,...,"ubicación_de_la_escuela-localidad_al_norte_del_ecuador,_expresada_en_grados,_minutos_y_segundos","ubicación_de_la_escuela-localidad_al_oeste_del_meridiano_de_greenwich,_expresada_en_grados","ubicación_de_la_escuela-localidad_al_norte_del_ecuador,_expresada_en_grados",x,y,CVEGEOMUN,address,latitud,longitud,geometry
clave_del_centro_de_trabajo,,,,,,,,,,,,,,,,,,,,,
01DES0040Q,14,1,MATUTINO,"SECUNDARIA GENERAL NUM. 39, LUIS DONALDO COLOS...",BASICA,SECUNDARIA,GENERAL,PUBLICO,FEDERAL TRANSFERIDO,1,...,21:57:32.18 N,-102.255060,21.958940,-102.255060,21.958940,01001,"CALLE PASEO DEL SENOR DEL ENCINO SUR 0, AGUASC...",21.958940,-102.255060,POINT (-102.25506 21.95894)
01DES0013T,20,1,MATUTINO,"SECUNDARIA GENERAL NUM. 24, JOSE GUADALUPE NAJ...",BASICA,SECUNDARIA,GENERAL,PUBLICO,FEDERAL TRANSFERIDO,1,...,21:50:32.53 N,-102.189539,21.842369,-102.189539,21.842369,01001,"AVENIDA REVOLUCION 0, AGUASCALIENTES, AGUASCAL...",21.842369,-102.189539,POINT (-102.18954 21.84237)
01KJN0280P,26,1,MATUTINO,PREESCOLAR COMUNITARIO,BASICA,INICIAL GENERAL,INICIAL INDIGENA,PUBLICO,FEDERAL,1,...,21:54:33.00 N,-102.381111,21.909167,-102.381111,21.909167,01005,"CALLE CEDROS 0, JESUS MARIA, AGUASCALIENTES, 2...",21.909167,-102.381111,POINT (-102.38111 21.90917)
01FEI0464N,39,5,CONTINUO,MODALIDAD NO ESCOLARIZADA JESUS MARIA,INICIAL,INICIAL,INICIAL NO ESCOLARIZADA,PUBLICO,FEDERAL TRANSFERIDO,1,...,21:57:40.92 N,-102.343472,21.961368,-102.343472,21.961368,01005,"NINGUNO NINGUNO 0, JESUS MARIA, AGUASCALIENTES...",21.961368,-102.343472,POINT (-102.34347 21.96137)
01DPR0065F,55,2,VESPERTINO,SOLIDARIDAD,BASICA,PRIMARIA,GENERAL,PUBLICO,FEDERAL TRANSFERIDO,1,...,21:57:12.73 N,-102.348620,21.953535,-102.348620,21.953535,01005,"CALLE LEONA VICARIO 0, JESUS MARIA, AGUASCALIE...",21.953535,-102.348620,POINT (-102.34862 21.95353)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32USU2374S,5547,4,DISCONTINUO,UNIDAD ACADEMICA DE HISTORA,SUPERIOR,LICENCIATURA,LICENCIATURA UNIVERSITARIA Y TECNOLOGICA,PUBLICO,AUTONOMO,32,...,22:46:5.33 N,-102.558296,22.768146,-102.558296,22.768146,32056,"AVENIDA PREPARATORIA 11, ZACATECAS, ZACATECAS,...",22.768146,-102.558296,POINT (-102.55830 22.76815)
32UBH0004S,5556,2,VESPERTINO,PREPARATORIA NUM. 4,MEDIA SUPERIOR,BACHILLERATO,BACHILLERATO GENERAL,PUBLICO,AUTONOMO,32,...,22:45:37.67 N,-102.577265,22.760463,-102.577265,22.760463,32056,"CALLE CARLOS LAZO 0, ZACATECAS, ZACATECAS, 980...",22.760463,-102.577265,POINT (-102.57726 22.76046)
32PSU0061S,5560,4,DISCONTINUO,UNIVERSIDAD IGNACIO CASTRO PEREZ,SUPERIOR,LICENCIATURA,LICENCIATURA UNIVERSITARIA Y TECNOLOGICA,PRIVADO,PRIVADO,32,...,22:45:2.12 N,-102.512830,22.750590,-102.512830,22.750590,32017,"CALLE DEL ANGEL 76, GUADALUPE, ZACATECAS, 9860...",22.750590,-102.512830,POINT (-102.51283 22.75059)


In [214]:
sigade_complete_version = good_final.append(corrected_all)

In [222]:
print(str(len(sigade_complete_version)) + ' out of a total ' + str(len(all_sigade_city)) + ' recovered with geocoding')


105377 out of a total 110321 recovered with geocoding


In [227]:
sigade_named = gpd.GeoDataFrame()
for c in cities.columns.unique():
    aup.log(f"{c} starting")
    query = f"SELECT * FROM metropolis.metro_list WHERE \"city\" LIKE \'{c}\'"
    mun_gdf = aup.gdf_from_query(query, geometry_col='geometry')
    # Iterates over municipality codes for each metropolitan area or capital
    sigade_temp = sigade_complete_version[sigade_complete_version['CVEGEOMUN'].isin(mun_gdf.CVEGEO)]
    sigade_temp['metropolis'] = str(c)
    sigade_named = sigade_named.append(sigade_temp)

/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [229]:
sigade_named.set_crs("EPSG:4326", inplace=True)
sigade_named.to_csv('sigade_complete_final.csv')
sigade_named.to_file('sigade_complete_final.geojson')

In [230]:
sigade_upload = gpd.read_file('/home/jovyan/work/notebooks/sigade_complete_final.geojson')

In [233]:
sigade_upload.drop(['ubicación_de_la_escuela-localidad_al_oeste_del_meridiano_de_greenwich,_expresada_en_grados,_minutos_y_segundos', 'ubicación_de_la_escuela-localidad_al_norte_del_ecuador,_expresada_en_grados,_minutos_y_segundos', 'ubicación_de_la_escuela-localidad_al_oeste_del_meridiano_de_greenwich,_expresada_en_grados', 'ubicación_de_la_escuela-localidad_al_norte_del_ecuador,_expresada_en_grados' ], axis =1, inplace =True)

In [234]:
sigade_upload.columns

Index(['clave_del_centro_de_trabajo', 'unnamed:_0', 'clave_del_turno',
       'nombre_del_turno', 'nombre_del_centro_de_trabajo', 'tipo_educativo',
       'nivel_educativo', 'servicio_educativo',
       'nombre_del_control_(público_o_privado)', 'tipo_de_sostenimiento',
       'clave_de_la_entidad_federativa', 'nombre_de_la_entidad',
       'clave_del_municipio_o_delegación', 'nombre_del_municipio_o_delegación',
       'clave_de_la_localidad', 'nombre_de_localidad', 'domicilio',
       'número_exterior', 'entre_la_calle', 'y_la_calle', 'calle_posterior',
       'colonia', 'nombre_de_la_colonia', 'código_postal', 'clave_lada',
       'teléfono', 'extensión_del_teléfono', 'página_web',
       'alumnos_total_(hombres)', 'alumnos_total_(mujeres)', 'alumnos_total',
       'docentes_total_(hombres)', 'docentes_total_(mujeres)',
       'docentes_total', 'aulas_en_uso', 'aulas_existentes',
       'tipo_de_localidad_u_-_urbano,_r_-_rural', 'x', 'y', 'CVEGEOMUN',
       'address', 'latitud', 'lon

In [235]:
aup.gdf_to_db_slow(sigade_upload, 'sigade_2020', 'denue', if_exists='append')